In [1]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import tensorflow.keras as k
#from sklearnex import patch_sklearn
#patch_sklearn()
import sklearn
import os
import numba
import time
import json
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns

In [2]:
final_word_embeddings=pd.read_json("final_word_embeddings.json")
final_data=pd.read_json("final_corpus_data.json")

In [9]:
#remove this
file=open("file_word_index.json","r")
word_index_rev=json.load(file)
file.close()
#
# Recalling the list of irrelevant words from the present context out of Glove
file=open("irrelevant_words.json","r")
irrelevant_words=json.load(file)
file.close()


# Chapter 3

In [4]:
final_data # Each number in the column represents the relative position and the word index of the particular word

,0,1,2,3,4,5,6,7,8,9
0,158,20885,1613,7733,6416,1780,404086,2281,2343,26926
1,400830,5572,3162,18102,210,5031,440,2772,0,0
2,6076,1188,1656,631,10,1171,282,5276,874,32277
3,18206,304,703,4928,327,2489,402304,1160,0,0
4,27493,20565,3468,61892,1752,4159,10409,180,0,0
...,...,...,...,...,...,...,...,...,...,...
26704,141,1379,4012,131387,0,0,0,0,0,0
26705,454,10,255,6614,0,0,0,0,0,0
26706,50717,766,0,0,0,0,0,0,0,0
26707,407,1366,5628,5261,6241,1475,7204,2513,0,0


## Preparing our Embeddings for further processing.

In [5]:
final_word_embeddings

,words,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,the,-0.038194,-0.244870,0.72812,-0.399610,0.083172,0.043953,-0.391410,0.334400,-0.57545,...,0.016215,-0.017099,-0.389840,0.87424,-0.72569,-0.51058,-0.520280,-0.14590,0.82780,0.270620
1,",",-0.107670,0.110530,0.59812,-0.543610,0.673960,0.106630,0.038867,0.354810,0.06351,...,0.349510,-0.722600,0.375490,0.44410,-0.99059,0.61214,-0.351110,-0.83155,0.45293,0.082577
2,.,-0.339790,0.209410,0.46348,-0.647920,-0.383770,0.038034,0.171270,0.159780,0.46619,...,-0.063351,-0.674120,-0.068895,0.53604,-0.87773,0.31802,-0.392420,-0.23394,0.47298,-0.028803
3,of,-0.152900,-0.242790,0.89837,0.169960,0.535160,0.487840,-0.588260,-0.179820,-1.35810,...,0.187120,-0.018488,-0.267570,0.72700,-0.59363,-0.34839,-0.560940,-0.59100,1.00390,0.206640
4,to,-0.189700,0.050024,0.19084,-0.049184,-0.089737,0.210060,-0.549520,0.098377,-0.20135,...,-0.131340,0.058617,-0.318690,-0.61419,-0.62393,-0.41548,-0.038175,-0.39804,0.47647,-0.159830
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404200,jlaw,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,0.000000
404201,'justice,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,0.000000
404202,'life-changer,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,0.000000
404203,'gobble,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,0.000000


First we need to reset our Word embedding too where the first word embedding i.e. 0 should represent "end of sequence" which should be a row entry here in embeddings matrix. 

In [7]:
final_word_embeddings.index=final_word_embeddings.index+1
final_word_embeddings

,words,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
1,the,-0.038194,-0.244870,0.72812,-0.399610,0.083172,0.043953,-0.391410,0.334400,-0.57545,...,0.016215,-0.017099,-0.389840,0.87424,-0.72569,-0.51058,-0.520280,-0.14590,0.82780,0.270620
2,",",-0.107670,0.110530,0.59812,-0.543610,0.673960,0.106630,0.038867,0.354810,0.06351,...,0.349510,-0.722600,0.375490,0.44410,-0.99059,0.61214,-0.351110,-0.83155,0.45293,0.082577
3,.,-0.339790,0.209410,0.46348,-0.647920,-0.383770,0.038034,0.171270,0.159780,0.46619,...,-0.063351,-0.674120,-0.068895,0.53604,-0.87773,0.31802,-0.392420,-0.23394,0.47298,-0.028803
4,of,-0.152900,-0.242790,0.89837,0.169960,0.535160,0.487840,-0.588260,-0.179820,-1.35810,...,0.187120,-0.018488,-0.267570,0.72700,-0.59363,-0.34839,-0.560940,-0.59100,1.00390,0.206640
5,to,-0.189700,0.050024,0.19084,-0.049184,-0.089737,0.210060,-0.549520,0.098377,-0.20135,...,-0.131340,0.058617,-0.318690,-0.61419,-0.62393,-0.41548,-0.038175,-0.39804,0.47647,-0.159830
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404201,jlaw,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,0.000000
404202,'justice,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,0.000000
404203,'life-changer,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,0.000000
404204,'gobble,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,0.000000


Now, we will add our extra word embedding i.e. Null Vector entry implying that it is null vector.

In [8]:
final_word_embeddings.loc[0]=["<NULL>"]+list(np.zeros(100))
final_word_embeddings

,words,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
1,the,-0.038194,-0.244870,0.72812,-0.399610,0.083172,0.043953,-0.391410,0.334400,-0.57545,...,0.016215,-0.017099,-0.389840,0.87424,-0.72569,-0.51058,-0.520280,-0.14590,0.82780,0.270620
2,",",-0.107670,0.110530,0.59812,-0.543610,0.673960,0.106630,0.038867,0.354810,0.06351,...,0.349510,-0.722600,0.375490,0.44410,-0.99059,0.61214,-0.351110,-0.83155,0.45293,0.082577
3,.,-0.339790,0.209410,0.46348,-0.647920,-0.383770,0.038034,0.171270,0.159780,0.46619,...,-0.063351,-0.674120,-0.068895,0.53604,-0.87773,0.31802,-0.392420,-0.23394,0.47298,-0.028803
4,of,-0.152900,-0.242790,0.89837,0.169960,0.535160,0.487840,-0.588260,-0.179820,-1.35810,...,0.187120,-0.018488,-0.267570,0.72700,-0.59363,-0.34839,-0.560940,-0.59100,1.00390,0.206640
5,to,-0.189700,0.050024,0.19084,-0.049184,-0.089737,0.210060,-0.549520,0.098377,-0.20135,...,-0.131340,0.058617,-0.318690,-0.61419,-0.62393,-0.41548,-0.038175,-0.39804,0.47647,-0.159830
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404202,'justice,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,0.000000
404203,'life-changer,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,0.000000
404204,'gobble,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,0.000000
404205,'wipe,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,0.000000


Embedding a new layer for our pre-trained Glove model in proper way and make it non trainable so that only the additionally added extra_words get trained, and the others stay just constant. For that purpose, we might have to split our data somewhere or some place and go from there. Let us see how we go from there.

**Proposed plan**

Here there are too many words in the original model. We can either go with the original model with added words as given here or proceed to chop off irrelevant words in the embeddings and make the final embedding model actually trainable.

If we recall, we have collected the words that are present in Glove but are not present in the present corpus. Of course, it is hopeful that the words given here in the present corpus is also the words that are assumed to repeat in the future.

Also this would reduce the computational load a lot. Though it is NOT RECOMMENDED to do this, since it completely eliminates the option of doing word similarity (via cosine similarity or something) to arrive at closest words and make a guess, it is handy and less expensive. (I have tried and it takes for ever to do the pre-processing functions on it.)

In [10]:
irrelevant_words

['czechoslovakia',
 'metes',
 'kapurthala',
 'punjab',
 'messias',
 'duclos',
 '3,516',
 'hausknecht',
 'i-985',
 'drumlins',
 'jugglers',
 'ss-panzer',
 'mogielnica',
 '45-5',
 'leatherwood',
 'molinier',
 'puriri',
 'butka',
 'marubeni',
 'tieling',
 'd&b',
 'nikolaev',
 'kibitz',
 'katzenberg',
 'aitc',
 'rhône',
 'wael',
 '192.5',
 '4.63',
 'delt',
 'sergeant-at-arms',
 'flukey',
 'queenfish',
 'shuozhou',
 'durbanville',
 'nonpaying',
 'mannitol',
 'hagiographers',
 '94.21',
 'post-world',
 '2,832',
 'monib',
 'splints',
 'kushida',
 '29,583',
 'yhs',
 'meštrović',
 'bmb',
 'finfoots',
 'imron',
 'euro343',
 'peekskill',
 'subsection',
 'prakasam',
 '9/11-style',
 '248.5',
 'iouri',
 'duplantier',
 '1,126',
 'travellers',
 'inniss',
 'nautiloids',
 'dous',
 'marne',
 'unrealized',
 'euro306',
 'andrewartha',
 'punkish',
 'mcdearmon',
 'sharabati',
 'giller',
 'wsou',
 'bedevil',
 'gostynin',
 'umea',
 'rabindranath',
 'mastersingers',
 'roosts',
 'siggins',
 'bordertown',
 'o-meth

In [23]:
final_word_embeddings.sort_index(inplace=True)
temp=final_word_embeddings.copy()
temp["org_index"]=temp.index
temp.index=temp["words"]
temp.drop("words",axis=1)

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,org_index
words,,,,,,,,,,,,,,,,,,,,,
<NULL>,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,...,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,0
the,-0.038194,-0.24487,0.72812,-0.39961,0.083172,0.043953,-0.391410,0.33440,-0.57545,0.087459,...,-0.017099,-0.389840,0.87424,-0.72569,-0.51058,-0.52028,-0.14590,0.82780,0.270620,1
",",-0.107670,0.11053,0.59812,-0.54361,0.673960,0.106630,0.038867,0.35481,0.06351,-0.094189,...,-0.722600,0.375490,0.44410,-0.99059,0.61214,-0.35111,-0.83155,0.45293,0.082577,2
.,-0.339790,0.20941,0.46348,-0.64792,-0.383770,0.038034,0.171270,0.15978,0.46619,-0.019169,...,-0.674120,-0.068895,0.53604,-0.87773,0.31802,-0.39242,-0.23394,0.47298,-0.028803,3
of,-0.152900,-0.24279,0.89837,0.16996,0.535160,0.487840,-0.588260,-0.17982,-1.35810,0.425410,...,-0.018488,-0.267570,0.72700,-0.59363,-0.34839,-0.56094,-0.59100,1.00390,0.206640,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
jlaw,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,...,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,404201
'justice,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,...,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,404202
'life-changer,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,...,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,404203


Now let us take away the words that aren't in our corpus vocabulary, given by the variable "irrelevant_words"

In [24]:
temp=temp.drop(irrelevant_words,axis=0).drop("words",axis=1)
temp

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,org_index
words,,,,,,,,,,,,,,,,,,,,,
<NULL>,0.00000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.00000,0.000000,0.00000,0
's,0.58854,-0.202500,0.734790,-0.68338,-0.19675,-0.180200,-0.391770,0.34172,-0.60561,0.63816,...,0.77649,0.12598,0.54779,-1.03160,0.064599,-0.375230,-0.94475,0.618020,0.39591,10
said,-0.13128,-0.452000,0.043399,-0.99798,-0.21053,-0.958680,-0.246090,0.48413,0.18178,0.47500,...,-0.62879,0.29098,0.13238,-0.70414,0.671280,-0.085462,-0.30526,-0.045495,0.56509,17
as,-0.32721,0.096446,0.342440,-0.44327,0.30535,-0.042016,-0.071235,-0.31036,-0.22557,-0.18100,...,-0.25332,-0.30730,0.51295,-0.56802,-0.212390,-0.397410,-0.38165,0.439940,0.24683,20
'',0.16478,0.170710,0.621110,-1.21010,-0.84063,0.218930,0.481230,-0.15044,0.36701,-0.20857,...,-0.56735,0.22621,0.47139,-0.48157,0.564380,0.340190,-0.44047,-0.105870,0.79509,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
jlaw,0.00000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.00000,0.000000,0.00000,404201
'justice,0.00000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.00000,0.000000,0.00000,404202
'life-changer,0.00000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.00000,0.000000,0.00000,404203


In this step we now create a fresh new index out of all this and assign it again a new column to the dataframe.

In [25]:
temp=temp.reset_index()
temp["new_index"]=temp.index

Now, let us look at the dataframe and see how it is going in order to make it working.

In [26]:
temp

,words,0,1,2,3,4,5,6,7,8,...,92,93,94,95,96,97,98,99,org_index,new_index
0,<NULL>,0.00000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.000000,0.000000,0.00000,0.000000,0.00000,0,0
1,'s,0.58854,-0.202500,0.734790,-0.68338,-0.19675,-0.180200,-0.391770,0.34172,-0.60561,...,0.12598,0.54779,-1.03160,0.064599,-0.375230,-0.94475,0.618020,0.39591,10,1
2,said,-0.13128,-0.452000,0.043399,-0.99798,-0.21053,-0.958680,-0.246090,0.48413,0.18178,...,0.29098,0.13238,-0.70414,0.671280,-0.085462,-0.30526,-0.045495,0.56509,17,2
3,as,-0.32721,0.096446,0.342440,-0.44327,0.30535,-0.042016,-0.071235,-0.31036,-0.22557,...,-0.30730,0.51295,-0.56802,-0.212390,-0.397410,-0.38165,0.439940,0.24683,20,3
4,'',0.16478,0.170710,0.621110,-1.21010,-0.84063,0.218930,0.481230,-0.15044,0.36701,...,0.22621,0.47139,-0.48157,0.564380,0.340190,-0.44047,-0.105870,0.79509,28,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25906,jlaw,0.00000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.000000,0.000000,0.00000,0.000000,0.00000,404201,25906
25907,'justice,0.00000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.000000,0.000000,0.00000,0.000000,0.00000,404202,25907
25908,'life-changer,0.00000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.000000,0.000000,0.00000,0.000000,0.00000,404203,25908
25909,'gobble,0.00000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.000000,0.000000,0.00000,0.000000,0.00000,404204,25909


## Processing Corpus Data

Now let us go back and replace our original word indexes inside the corpus with newly generated word indexes after elimination of unwnated words. For that we need to create a number map or index mapping, as a dictionary and, that is given in the above table. Use it to remove entries that depended on golve vector embedding.

**Please Note** 

It didn't occur to me that we could have done all that before we even put word indexes into the data. So, I didn't do it. But, now here I do it. For that let me create a dictionary map first.

In [27]:
index_remap=dict(zip(list(temp["org_index"]),list(temp["new_index"])))

In [29]:
final_data_clean=final_data.replace(to_replace=index_remap)
final_data_clean.head()

,0,1,2,3,4,5,6,7,8,9
0,64,11298,1244,5504,4695,1376,25791,1757,1802,12973
1,22535,4123,2417,10371,105,3748,298,2142,0,0
2,4472,897,1280,459,1,884,159,3926,658,14109
3,10410,179,516,3676,199,1918,24009,874,0,0
4,13112,11202,2658,17546,1354,3157,7032,83,0,0


In [30]:
final_data.head()

,0,1,2,3,4,5,6,7,8,9
0,158,20885,1613,7733,6416,1780,404086,2281,2343,26926
1,400830,5572,3162,18102,210,5031,440,2772,0,0
2,6076,1188,1656,631,10,1171,282,5276,874,32277
3,18206,304,703,4928,327,2489,402304,1160,0,0
4,27493,20565,3468,61892,1752,4159,10409,180,0,0


In [31]:
final_data_clean.to_json("final_data_clean.json")
# Saving the data to reload in next chapter

In [32]:
final_data_clean

,0,1,2,3,4,5,6,7,8,9
0,64,11298,1244,5504,4695,1376,25791,1757,1802,12973
1,22535,4123,2417,10371,105,3748,298,2142,0,0
2,4472,897,1280,459,1,884,159,3926,658,14109
3,10410,179,516,3676,199,1918,24009,874,0,0
4,13112,11202,2658,17546,1354,3157,7032,83,0,0
...,...,...,...,...,...,...,...,...,...,...
26704,50,1049,3046,20008,0,0,0,0,0,0
26705,310,1,139,4818,0,0,0,0,0,0
26706,16633,571,0,0,0,0,0,0,0,0
26707,270,1038,4167,3915,4576,1126,5192,1937,0,0


**Caution**

We have to do one more thing here. We have to attach our NULL Vector at the end of each sequence indicating that it has reached the end. Here for maximum length sequences it hasn't been attached yet. So, we have to do it just before doing anything else.

**IMPORTANT NOTE**

DO NOT directly map word index. We have morphed it to reset the word index.  So, while reconverting it back, we have to take care not to upset it. We have to reverse morph the values (after appropriate reduction of 1 for our padding word and its vector) and then search for it.


In [34]:
final_word_embeddings=temp

In [35]:
final_word_embeddings.to_json("final_word_embeddings_clean.json")

In [36]:
final_word_embeddings

,words,0,1,2,3,4,5,6,7,8,...,92,93,94,95,96,97,98,99,org_index,new_index
0,<NULL>,0.00000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.000000,0.000000,0.00000,0.000000,0.00000,0,0
1,'s,0.58854,-0.202500,0.734790,-0.68338,-0.19675,-0.180200,-0.391770,0.34172,-0.60561,...,0.12598,0.54779,-1.03160,0.064599,-0.375230,-0.94475,0.618020,0.39591,10,1
2,said,-0.13128,-0.452000,0.043399,-0.99798,-0.21053,-0.958680,-0.246090,0.48413,0.18178,...,0.29098,0.13238,-0.70414,0.671280,-0.085462,-0.30526,-0.045495,0.56509,17,2
3,as,-0.32721,0.096446,0.342440,-0.44327,0.30535,-0.042016,-0.071235,-0.31036,-0.22557,...,-0.30730,0.51295,-0.56802,-0.212390,-0.397410,-0.38165,0.439940,0.24683,20,3
4,'',0.16478,0.170710,0.621110,-1.21010,-0.84063,0.218930,0.481230,-0.15044,0.36701,...,0.22621,0.47139,-0.48157,0.564380,0.340190,-0.44047,-0.105870,0.79509,28,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25906,jlaw,0.00000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.000000,0.000000,0.00000,0.000000,0.00000,404201,25906
25907,'justice,0.00000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.000000,0.000000,0.00000,0.000000,0.00000,404202,25907
25908,'life-changer,0.00000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.000000,0.000000,0.00000,0.000000,0.00000,404203,25908
25909,'gobble,0.00000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.000000,0.000000,0.00000,0.000000,0.00000,404204,25909


Our Word embedding is finally complete along with appropriate reset word vectors, and also word encodings properly reset. Now it is time to load into our model, by simply resetting our embedding model to share the sequential indices.


Notice here that at the end of every sequence there is a zero and corresponding zero vector embedding indicating the end of the sequence. We haven't yet added that pad to our original dataframe of corpus

In [286]:

import tensorflow
from tensorflow.keras.layers import Embedding
vocab_size2=f_embed2.shape[0] #We can choose to have the most frequent words as tokens or simply leave it to be the actual vocab size
embedding_dimensions=100 # We will retain our original along with newly added words
length_of_sequence2=final_data.shape[1] +2 #This is very crucial and needs to be appropriate or sometimes chopping off for extra long sequences might not happen.
#embedding_matrix2=np.zeros((vocab_size+len(),embedding_dimensions))
#embedding_matrix2=f_embed2.to_numpy()


embedding_layer=Embedding (vocab_size2,
                           embedding_dimensions,
                           embeddings_initializer=tensorflow.initializers.Constant(f_embed2.to_numpy()),
                           trainable=True,
                          input_length=length_of_sequence)



#embedding_layer(final_data.to_numpy()) #let us give some random entry of 46th article of the corpus)
from tensorflow.keras.layers import Input
input_sequences=Input(shape=(None,),dtype="int64")
k=embedding_layer(input_sequences)

**Note**
This code is given as markdown for my own future references.


import tensorflow
from tensorflow.keras.layers import Embedding
vocab_size=f_embed.shape[0] #We can choose to have the most frequent words as tokens or simply leave it to be the actual vocab size
embedding_dimensions=100 # We will retain our original along with newly added words
length_of_sequence=final_data.shape[1] +2 #This is very crucial and needs to be appropriate or sometimes chopping off for extra long sequences might not happen.
#embedding_matrix2=np.zeros((vocab_size+len(),embedding_dimensions))
#embedding_matrix2=f_embed2.to_numpy()



embedding_layer=Embedding (vocab_size,
                           embedding_dimensions,
                           embeddings_initializer=tensorflow.initializers.Constant(f_embed.to_numpy()),
                           trainable=True,
                          input_length=length_of_sequence)

#embedding_layer(final_data.to_numpy()) #let us give some random entry of 46th article of the corpus)
from tensorflow.keras.layers import Input
input_sequences=Input(shape=final_data.shape,dtype="int64")
embedding_layer(input_sequences)

In [282]:
k

<KerasTensor: shape=(None, 26709, 10, 100) dtype=float32 (created by layer 'embedding_30')>

## End of Chapter 3

# Constructing Network Layers

Now Since our Embedding layers are ready and giving us proper inputs, let us construct a Neural Network layers in a Keras Model Class

In [290]:
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Bidirectional

lstm_out = 50

#k=SpatialDropout1D(0.4)(k)
#k=Bidirectional(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))(k)
k=LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2,return_sequences=True)(k)
k=Dense(12,activation='relu')(k)
ouputs=Dense(1,activation="softmax")
model=Model(inputs=input_sequences,outputs=outputs)
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

NotImplementedError: Cannot convert a symbolic Tensor (lstm_3/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported